In [19]:
import requests
from bs4 import BeautifulSoup
import time
import hashlib
from datetime import datetime
import logging
import json
import re
import os



In [20]:
class Schedule2DriveMonitor:
    def __init__(self, permit_number, birth_month, birth_day, birth_year, input_login_state, check_interval=300):
        """
        Initialize the monitor for Schedule2Drive
        
        Args:
            permit_number: Permit/DL/ID number
            birth_month: Two digit birth month
            birth_day: Two digit birth day 
            birth_year: Four digit birth year
            input_login_state: Two letter state code
            check_interval: Time between checks in seconds (default 5 minutes)
        """
        self.base_url = "https://www.schedule2drive.com"
        self.login_url = f"{self.base_url}/index.php"
        self.calendar_base_url = f"{self.base_url}/student.php"
        
        # Format all inputs as strings with leading zeros where needed
        self.login_data = {
            "inputLoginPermit": str(permit_number),
            "inputLoginMonth": str(birth_month).zfill(2),
            "inputLoginDay": str(birth_day).zfill(2),
            "inputLoginYear": str(birth_year),
            "inputLoginState": input_login_state.upper(),
            "forceUI": "d",  # Desktop UI
            "_event[submitStudent]": "_event[submitStudent]"  # This matches the JavaScript exactly
        }
        
        self.check_interval = check_interval
        self.session = requests.Session()
        self.previous_hash = None
        
        # Set up logging
        logging.basicConfig(
            filename='schedule2drive_monitor.log',
            level=logging.INFO,
            format='%(asctime)s - %(message)s'
        )
        
        # Set headers to mimic a browser more accurately
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.9',
            'Origin': 'https://www.schedule2drive.com',
            'Referer': 'https://www.schedule2drive.com/index.php',
            'Cache-Control': 'no-cache',
            'Pragma': 'no-cache',
            'Sec-Fetch-Site': 'same-origin',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-User': '?1',
            'Sec-Fetch-Dest': 'document',
            'Upgrade-Insecure-Requests': '1'
        })

    def _save_debug_html(self, response, filename):
        """Save HTML response for debugging"""
        with open(os.path.join('..', 'debug', filename), 'w', encoding='utf-8') as f:
            f.write(response.text)
        logging.info(f"Saved debug HTML to {filename}")
    
    def login(self):
        """Perform login and return True if successful"""
        try:
            # First get the login page to get session cookie
            response = self.session.get(self.login_url)
            response.raise_for_status()
            self._save_debug_html(response, 'login_page.html')
            
            # Get PHPSESSID cookie from the response
            cookies = self.session.cookies.get_dict()
            logging.info(f"Initial cookies: {cookies}")
            
            # Parse the page to find the form and any hidden fields
            soup = BeautifulSoup(response.text, 'html.parser')
            form = soup.find('form', {'name': 'StudentLoginForm', 'id': 'StudentLoginForm'})
            
            if form:
                # Add any hidden fields to login_data
                for hidden in form.find_all('input', {'type': 'hidden'}):
                    if hidden.get('name') and hidden.get('value'):
                        self.login_data[hidden['name']] = hidden['value']
            
            # Add screen resolution as hidden field
            self.login_data['screenRes'] = '1920x1080'
            
            # Log the data we're about to send
            logging.info(f"Submitting login data: {self.login_data}")
            
            # Submit the form
            response = self.session.post(
                self.login_url,
                data=self.login_data,
                allow_redirects=True
            )
            
            response.raise_for_status()
            self._save_debug_html(response, 'login_response.html')
            
            # Log response details
            logging.info(f"Login response URL: {response.url}")
            logging.info(f"Login response status: {response.status_code}")
            logging.info(f"Login response headers: {dict(response.headers)}")
            logging.info(f"Cookies after login: {self.session.cookies.get_dict()}")
            
            if self._verify_login(response):
                logging.info("Login successful")
                # Immediately try to load student page
                student_response = self.session.get(
                    self.calendar_base_url,
                    headers={'Referer': response.url}
                )
                self._save_debug_html(student_response, 'immediate_student_page.html')
                return True
            else:
                logging.error("Login failed - Invalid credentials or form submission")
                return False
                
        except requests.exceptions.RequestException as e:
            logging.error(f"Login failed due to request error: {str(e)}")
            return False
    
    def _verify_login(self, response):
        """Verify if login was successful"""
        # Save response for debugging
        self._save_debug_html(response, 'verification_page.html')
        
        # Check if we're redirected to student page or have success indicators
        success_indicators = [
            "student.php" in response.url,
            "logout" in response.text.lower(),
            "welcome" in response.text.lower() and "student" in response.text.lower(),
            "schedule" in response.text.lower() and "calendar" in response.text.lower()
        ]
        
        return any(success_indicators)
    
    def _get_calendar_url(self):
        """Get the calendar URL with current session info"""
        try:
            # First try to get the student page
            response = self.session.get(self.calendar_base_url)
            response.raise_for_status()
            self._save_debug_html(response, 'student_page.html')
            
            logging.info(f"Calendar page URL: {response.url}")
            
            # If we got redirected back to login, we're not authenticated
            if "index.php" in response.url:
                logging.error("Not authenticated - redirected to login page")
                return None
                
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Look for various possible calendar links
            calendar_selectors = [
                ('a', {'href': re.compile(r'sessCal=1')}),
                ('a', {'href': re.compile(r'calendar')}),
                ('a', {'href': re.compile(r'schedule')}),
                ('a', {'class': re.compile(r'calendar')}),
                ('div', {'class': re.compile(r'calendar')})
            ]
            
            # Try each selector
            for tag, attrs in calendar_selectors:
                element = soup.find(tag, attrs)
                if element:
                    if element.name == 'a' and element.get('href'):
                        full_url = f"{self.base_url}/{element['href'].lstrip('/')}"
                        logging.info(f"Found calendar URL using selector {tag}, {attrs}: {full_url}")
                        return full_url
                    else:
                        logging.info(f"Found calendar element using selector {tag}, {attrs}")
                        return self.calendar_base_url
            
            # If we can't find the calendar link but we're on the student page, use the base URL
            if "student.php" in response.url:
                logging.info("Using student page as calendar URL")
                return self.calendar_base_url
            
            logging.error("Could not find calendar link")
            return None
            
        except Exception as e:
            logging.error(f"Error getting calendar URL: {str(e)}")
            return None
    
    def get_page_content(self):
        """Fetch the calendar page content"""
        calendar_url = self._get_calendar_url()
        if not calendar_url:
            return None
            
        try:
            response = self.session.get(calendar_url)
            response.raise_for_status()
            self._save_debug_html(response, 'calendar_page.html')
            return response.text
        except requests.exceptions.RequestException as e:
            logging.error(f"Failed to fetch calendar page: {str(e)}")
            return None
    
    def get_content_hash(self, content):
        """Generate a hash of the relevant content"""
        soup = BeautifulSoup(content, 'html.parser')
        
        # Remove elements that change but aren't relevant
        for element in soup.select('script, link, meta'):
            element.decompose()
            
        # Focus on the calendar content or main content area
        calendar_content = soup.find('div', {'class': ['calendar', 'mainArea', 'textBoxArea']})
        if calendar_content:
            return hashlib.md5(str(calendar_content).encode('utf-8')).hexdigest()
        return hashlib.md5(str(soup).encode('utf-8')).hexdigest()
    
    def check_for_changes(self):
        """Check if the calendar has changed"""
        if not self.login():
            return False
            
        content = self.get_page_content()
        if not content:
            return False
        
        current_hash = self.get_content_hash(content)
        
        if self.previous_hash is None:
            self.previous_hash = current_hash
            logging.info("Initial calendar hash recorded")
            return False
        
        if current_hash != self.previous_hash:
            self.previous_hash = current_hash
            return True
        
        return False
    
    def start_monitoring(self):
        """Start the monitoring loop"""
        logging.info("Starting Schedule2Drive monitor")
        
        while True:
            if self.check_for_changes():
                logging.info("Change detected in calendar availability!")
                # Add notification logic here
                
            time.sleep(self.check_interval)



In [21]:
# Example config

# monitor_config = {
#     "permit_number": "YOUR_PERMIT_NUMBER",
#     "birth_month": "MM",
#     "birth_day": "DD",
#     "birth_year": "YYYY",
#     "input_login_state": "WA"
#     "check_interval": 300,
# }


with open(os.path.join('..', 'data', 'config.json'), 'r') as f:
    monitor_config = json.load(f)

monitor = Schedule2DriveMonitor(**monitor_config)
monitor.start_monitoring()

KeyboardInterrupt: 